In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report, confusion_matrix, matthews_corrcoef, cohen_kappa_score, precision_recall_curve, auc
import numpy as np
import pandas as pd
import json
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [12]:
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

In [8]:
with open("absa_dataset_combined.json", "r", encoding="utf-8") as f:
    entries = json.load(f)

In [9]:
entries = [e for e in entries if all(k in e for k in ["text", "aspect", "sentiment"])]

In [13]:
# ========== Label Mapping ==========
label_map = {"positive": 0, "neutral": 1, "negative": 2}
id2label  = {v: k for k, v in label_map.items()}

In [14]:
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

In [2]:

# ========== Custom Dataset ==========
class ABSADataset(Dataset):
    def __init__(self, entries, tokenizer, label_map, max_len=128):
        self.entries = entries
        self.tokenizer = tokenizer
        self.label_map = label_map
        self.max_len = max_len

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        item = self.entries[idx]
        text = item["text"]
        aspect = item["aspect"]
        label = self.label_map[item["sentiment"]]

        encoded = self.tokenizer(
            f"{text} [ASP] {aspect}",
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )

        return {
            "input_ids": encoded["input_ids"].squeeze(),
            "attention_mask": encoded["attention_mask"].squeeze(),
            "labels": torch.tensor(label)
        }

In [15]:
dataset = ABSADataset(entries, tokenizer, label_map)

In [17]:
from sklearn.model_selection import train_test_split
train_entries, val_entries = train_test_split(entries, test_size=0.1, stratify=[e["sentiment"] for e in entries], random_state=42)
train_dataset = ABSADataset(train_entries, tokenizer, label_map)
val_dataset = ABSADataset(val_entries, tokenizer, label_map)

In [18]:
model = XLMRobertaForSequenceClassification.from_pretrained(
    "xlm-roberta-base",
    num_labels=3,
    id2label=id2label
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Error while downloading from https://huggingface.co/xlm-roberta-base/resolve/main/model.safetensors: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2536)
Trying to resume download...
Error while downloading from https://huggingface.co/xlm-roberta-base/resolve/main/model.safetensors: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2536)
Trying to resume download...
Error while downloading from https://huggingface.co/xlm-roberta-base/resolve/main/model.safetensors: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2536)
Trying to resume download...
Error while downloading from https://huggingface.co/xlm-roberta-base/resolve/main/model.safetensors

In [19]:
training_args = TrainingArguments(
    output_dir="./absa-checkpoints",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    report_to="none"
)

c:\Users\ordinateur5\Documents\figo\projects\modelisation-fine-tuning-xlm\venv\lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    report = classification_report(labels, preds, target_names=label_map.keys(), output_dict=True)
    mcc = matthews_corrcoef(labels, preds)
    kappa = cohen_kappa_score(labels, preds)
    conf_mat = confusion_matrix(labels, preds)

    print("\nConfusion Matrix:\n", conf_mat)
    print("\nClassification Report:\n", classification_report(labels, preds, target_names=label_map.keys()))
    print(f"MCC: {mcc:.3f}, Kappa: {kappa:.3f}")

    return {
        "accuracy": report["accuracy"],
        "f1_positive": report["positive"]["f1-score"],
        "f1_neutral": report["neutral"]["f1-score"],
        "f1_negative": report["negative"]["f1-score"],
        "mcc": mcc,
        "kappa": kappa
    }


In [21]:
# ========== Trainer ==========
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

C:\Users\ordinateur5\AppData\Local\Temp\ipykernel_2432\72840927.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1 Positive,F1 Neutral,F1 Negative,Mcc,Kappa
1,0.895500,0.890297,0.513205,0.678302,0.000000,0.000000,0.000000,0.000000



Confusion Matrix:
 [[719   0   0]
 [ 94   0   0]
 [588   0   0]]

Classification Report:
               precision    recall  f1-score   support

    positive       0.51      1.00      0.68       719
     neutral       0.00      0.00      0.00        94
    negative       0.00      0.00      0.00       588

    accuracy                           0.51      1401
   macro avg       0.17      0.33      0.23      1401
weighted avg       0.26      0.51      0.35      1401

MCC: 0.000, Kappa: 0.000


c:\Users\ordinateur5\Documents\figo\projects\modelisation-fine-tuning-xlm\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ordinateur5\Documents\figo\projects\modelisation-fine-tuning-xlm\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ordinateur5\Documents\figo\projects\modelisation-fine-tuning-xlm\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `ze

KeyError: "The `metric_for_best_model` training argument is set to 'eval_f1', which is not found in the evaluation metrics. The available evaluation metrics are: ['eval_loss', 'eval_accuracy', 'eval_f1_positive', 'eval_f1_neutral', 'eval_f1_negative', 'eval_mcc', 'eval_kappa']. Consider changing the `metric_for_best_model` via the TrainingArguments."

In [ ]:
# ========== Final Evaluation ==========
eval_results = trainer.evaluate()
print("\nFinal Evaluation Results:", eval_results)